In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
%matplotlib inline

In [3]:
from IPython.display import SVG
from urllib.request import urlretrieve
import zipfile
import collections

### Todo

###### Få tal på categories = 1145
###### Inkluder categories i dataset
###### Lav embedding af categories, som kan bruges som input til netværket
###### Ændre i netværk/loss-function

# Create Subset of Dataset

## Load in the dataset from csv files

In [4]:
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_Checkins.csv', sep=',', names=checkin_cols, encoding='latin-1')

venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\TIST2015_POIs.csv', sep=',', names=venue_cols, encoding='latin-1')


C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
print(pois)

                           poi_id   latitude   longitude           category  \
0                             Pid   Latitude  Longtitude           Category   
1        3fd66200f964a52000e71ee3  40.733596  -74.003139          Jazz Club   
2        3fd66200f964a52000e81ee3  40.758102  -73.975734                Gym   
3        3fd66200f964a52000ea1ee3  40.732456  -74.003755  Indian Restaurant   
4        3fd66200f964a52000ec1ee3  42.345907  -71.087001  Indian Restaurant   
...                           ...        ...         ...                ...   
3680122  5237865c498e89110c1d03e7  40.154444   26.410847        Comedy Club   
3680123  5237867411d2a1e910744c81  35.340099   33.309328     Home (private)   
3680124  5237879111d216bab10e9e09  -1.404065  -48.453742     Home (private)   
3680125  52378c24498ea9502baf2716   3.425155   -76.54501     Sandwich Place   
3680126  5237920a11d2173b20b50d4b  -6.129426  106.650974              Plane   

        country_code  
0            Country  
1    

## Function that extracts a subset

In [12]:
# Create a subset of a dataset with size n users
def create_checkin_subset(checkins_table, n):
    subset = np.array([[0,0,0,0],[0,0,0,0]])
    checkins_100_users = checkins_table['user_id'].sample(n=n, random_state=1)
    for i in checkins_100_users:
        subset = np.append(subset, np.asarray(checkins_table[checkins_table['user_id'] == i].values), 0)
    return pd.DataFrame(subset[2::], columns=['user_id', 'poi_id', 'timestamp', 'timezone'])

### Calling the function

In [187]:
# Initialize the subset method
checkins_100_users = create_checkin_subset(checkins, 25)
checkin_data = checkins_100_users.merge(pois, on='poi_id')
df = checkin_data.set_index('user_id').poi_id.str.get_dummies(',')
df = df.groupby('user_id').max()

In [229]:
checkin_data_no_duplicates = checkin_data.copy()
checkin_data_no_duplicates.drop_duplicates(subset ="poi_id",
                     keep = 'first', inplace = True)
checkin_data_no_duplicates = pd.DataFrame(checkin_data_no_duplicates, columns = ['poi_id', 'category'])

In [230]:
print(checkin_data_no_duplicates)

                        poi_id           category
0     4ef5d479f9ab2e6680a7fdda           Building
37    4e621c8d62e13e3bce7bb58d              Diner
38    4cb0fb5af2dbef3b6aae79e5              Plaza
90    5110b08de4b0d12c384d4e0b              Track
91    4d446c883616b60c0435e4c2           Hospital
...                        ...                ...
9279  4dde6ed4fa769803ef914cb5                Bar
9281  4c33694866e40f4727f3c78b         Restaurant
9282  4fdd8d30e4b01dfebfe1201b             Bakery
9283  5130f6b6e4b090128d2196c3  Convenience Store
9285  4f26bec6e4b0d10db2bb003c           Building

[2896 rows x 2 columns]


### Extract categorical data

In [278]:
categories = pd.DataFrame(checkin_data, columns=['category'])
categories.drop_duplicates(subset ="category",
                     keep = 'first', inplace = True)
category_length = len(categories)
categories_numpy = categories.to_numpy()

In [ ]:
print(category_length)
print(categories_numpy)

In [282]:
def one_hot_encode(ground_truth, lst):
    result = []
    for category in lst:
        oh_encoding = np.zeros(len(ground_truth))
        if category in ground_truth:
            print(category)
            index = np.where(ground_truth == category)[0][0]
            
            #Get index og category, and insert 1 into the vector.
            result.append(index)
    return result
        
        

In [ ]:
test = one_hot_encode(categories_numpy, categories_numpy)
print(test)

## Extracting all of the users and the pois

In [235]:
listofusers = pd.DataFrame(checkin_data, columns= ['user_id']).groupby('user_id').max().sample(frac=1)
listofpois = pd.DataFrame(checkin_data, columns= ['poi_id', 'latitude', 'longitude']).groupby('poi_id').max().sample(frac=1)

userarray = listofusers.index.to_numpy()
poiarray = listofpois.index.to_numpy()

userdataframe = pd.DataFrame(userarray, columns = ['Users'])
poidataframe = pd.DataFrame(poiarray, columns = ['Poi'])

In [203]:
print(len(userdataframe))

25


In [ ]:
#REDUNDANT, i think

rows_list = []
for i in range(len(poidataframe)):
    longitude = listofpois[listofpois.loc[i, "Poi"]]
    rows_list.append(dict1)

latlong = pd.DataFrame(rows_list) 
    

In [236]:
dot = userdataframe.merge(poidataframe, how='cross')

Define the model

In [237]:
print(listofpois)

                           latitude   longitude
poi_id                                         
4ba72bcaf964a5202d8439e3  35.659598  139.698415
4bda5640c79cc928889c7ee9  35.460382  139.628763
4b18bed2f964a5207dd523e3  35.702309  139.745032
4c7f41fe171b224b106d10b1  35.701256  139.770825
4bca0246fb84c9b66d251c3e  -6.166819  106.802948
...                             ...         ...
4a823f8af964a52001f91fe3   41.94733  -87.653695
4b5bab10f964a520e00e29e3  35.689718  139.699252
4b466ff5f964a520e82026e3   41.88573   -87.62796
51936834498ea46fc3c9f9b7  35.644116  139.748755
4b86747ef964a520cc8a31e3  35.452088  139.630673

[2896 rows x 2 columns]


In [206]:
print([poidataframe.loc[0, "Poi"]])
print(listofpois.loc['4c576c0ccc96c9b6a70f792e']['latitude'])

['4e3e05818877b00cfc504604']
37.843275


In [238]:
rows_list = []
for i in range(len(dot)):
    temp = dot.loc[i, "Poi"]
    latitude = listofpois.loc[temp]['latitude']
    longitude = listofpois.loc[temp]['longitude']
    dict1 = {'latitude':latitude, 'longitude':longitude}
    rows_list.append(dict1)
    #latitude = poiarray[i]
latlong = pd.DataFrame(rows_list)

In [133]:
print(latlong)

        latitude   longitude
0      35.387608  139.492374
1      19.446002  -99.162386
2      37.785255   29.084757
3      14.554799  121.049209
4      38.269416  140.925193
...          ...         ...
72395  33.418941 -111.925836
72396  37.476516  126.981641
72397  22.285136  114.157327
72398  35.658900  139.703264
72399  35.353181  139.531324

[72400 rows x 2 columns]


## Creating dataset

In [239]:
userdot = pd.DataFrame(dot, columns= ['Users'])
latlong['latitude'] = pd.to_numeric(latlong['latitude'])
latlong['longitude'] = pd.to_numeric(latlong['longitude'])
dataset = pd.concat([userdot, latlong], axis=1)

In [277]:
print(len(categories_numpy))

72400


## Extracting ground_truth from incidence matrix

In [284]:
rows_list = []
category_list = []
for i in range(len(dot)):
    temp = df[dot.loc[i, "Poi"]][dot.loc[i, "Users"]]
    dict1 = {'ground_truth':float(temp)}
    rows_list.append(dict1)
    #Extract category from the list
    category = checkin_data_no_duplicates.loc[checkin_data_no_duplicates['poi_id'] == dot.loc[i, "Poi"]]
    cat = category['category']
    index = np.where(categories_numpy == [cat])[0][0]
    category_list.append(index)
    
    
    

#category_label = 
groundtruth = pd.DataFrame(rows_list)
#result = pd.concat([dot, groundtruth], axis=1)

In [286]:
categories = pd.DataFrame(category_list, columns=['category'])
datasetst = pd.concat([dataset, categories], axis=1)

In [287]:
print(datasetst)

       Users   latitude   longitude  category
0      43298  35.659598  139.698415       139
1      43298  35.460382  139.628763        18
2      43298  35.702309  139.745032        86
3      43298  35.701256  139.770825        94
4      43298  -6.166819  106.802948        18
...      ...        ...         ...       ...
72395   9765  41.947330  -87.653695        86
72396   9765  35.689718  139.699252        86
72397   9765  41.885730  -87.627960        86
72398   9765  35.644116  139.748755        95
72399   9765  35.452088  139.630673        18

[72400 rows x 4 columns]


In [328]:
dataset_numpy = datasetst.to_numpy()
labels_numpy = groundtruth.to_numpy()
categories_numpy = categories.to_numpy()
x_train, x_test, y_train, y_test = train_test_split(dataset_numpy, labels_numpy, test_size=0.05, random_state=0)

x_train_df = pd.DataFrame(x_train, columns=['User','Latitude','Longitude', '0'])
x_test_df = pd.DataFrame(x_test, columns=['User','Latitude','Longitude', '0'])
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)

#Dataset with Users
dataset1_df = pd.DataFrame(x_train_df['User'])

#Dataset with Poi's
dataset2_df = pd.DataFrame(x_train_df[['Latitude', 'Longitude']])

dataset3_df = pd.DataFrame(x_train_df[['0']])


dataset1 = tf.convert_to_tensor(
    dataset1_df, dtype=None, dtype_hint=None, name=None)
dataset2 = tf.convert_to_tensor(
    dataset2_df, dtype=None, dtype_hint=None, name=None)
dataset3 = tf.convert_to_tensor(
    dataset3_df, dtype='int64', dtype_hint=None, name=None)
labels = tf.convert_to_tensor(
    y_train_df, dtype=None, dtype_hint=None, name=None)

In [329]:
print(dataset3)

tf.Tensor(
[[  2]
 [ 43]
 [117]
 ...
 [176]
 [  0]
 [ 86]], shape=(68780, 1), dtype=int64)


In [356]:
class EmbModel(Model):
    def __init__(self, poilength, useridlength, category_length):
        super(EmbModel, self).__init__()
        self.d_steps = 1
        self.poilength = poilength
        self.useridlength = useridlength
        self.category_length = category_length
        self.model = self.init_model()
        print(self.useridlength)
        print(self.poilength)
        
    def call(self, inputs):
        return
    
    def init_model(self):
        poi_latitude_input = keras.Input(shape=(1,), name='poi_latitude')
        poi_longitude_input = keras.Input(shape=(1,), name='poi_longitude')
        poi_concat_input = tf.keras.layers.Concatenate(axis=-1)([poi_latitude_input, poi_longitude_input])
        #input_length:  #This is the length of input sequences, as you would define for any input layer of a Keras model. 
                        #For example, if all of your input documents are comprised of 1000 words, this would be 1000
        #input_dim: 
                        #This is the size of the vocabulary in the text data. 
                        #For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
        poi_dense = layers.Dense(128)(poi_concat_input)
        poi_reshape = layers.Reshape((1, 128))(poi_dense)
        
        category_input = keras.Input(shape=(1), name='category_input')
        category_emb = layers.Embedding(self.category_length, 128)(category_input)
        
        category_concat = tf.keras.layers.Concatenate(axis=-1)([category_emb, poi_reshape])
    
        user_input = keras.Input(shape=(1,), name='user_id')
        #user_dense = keras.layers.Dense(2)(user_input)
        user_emb = layers.Dense(256)(user_input)
        user_reshape = layers.Reshape((1, 256))(user_emb)
        
                                    
        dot = layers.Dot(axes=(2))([category_concat, user_reshape])
        
        
        model = Model([category_input, [poi_latitude_input, poi_longitude_input], user_input], dot)
        model.summary()
        return model
    
    def compile_model(self, optimizer):
        super(EmbModel, self).compile(run_eagerly=True)
        self.optimizer = optimizer
        
    def train_step(self, data):
        if len(data) == 3:
            real_data, labels, sample_weight = data
        else:
            sample_weight = None
            real_data, labels = data
        user_data = real_data[0]
        latlong_data = real_data[1]
        cat_data = real_data[2]
        print(user_data)
        print(latlong_data)
        print(cat_data)
        
        for i in range(self.d_steps):
            
            with tf.GradientTape() as tape:
                
                #print(latlong_data[0])
                #print(latlong_data[1])
                #print(user_data)
                
                dotproduct = self.model([cat_data, [latlong_data[0], latlong_data[1]], user_data])
                #print(dotproduct)
                # Loss function = ||S-GroundTruth|| 
                loss = tf.math.abs(tf.subtract(tf.cast(dotproduct, tf.float64), labels))
                #print(loss)
            d_gradient = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(d_gradient, self.model.trainable_variables))
        return {'loss': loss}
    
    def predict_step(self, data):
        sample_weight = None
        real_data = data[0]
        user_data = real_data[0]
        latlong_data = real_data[1]
        return self.model([[latlong_data[0], latlong_data[1]], user_data])
        


In [357]:
print(len(userdataframe))

25


In [358]:
model = EmbModel(100, 1000, len(categories_numpy)+1)

Model: "functional_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
poi_latitude (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
poi_longitude (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate_39 (Concatenate)    (None, 2)            0           poi_latitude[0][0]               
                                                                 poi_longitude[0][0]              
__________________________________________________________________________________________________
category_input (InputLayer)     [(None, 1)]          0                                

In [359]:
import sys
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)

model.compile(
    optimizer
)

#Train_data, [dataset.user_id, dataset.poi_id]. Label: ground_truth
model.fit([dataset1, dataset2, dataset3], labels, epochs = 20, batch_size=2)

Epoch 1/20
Tensor("IteratorGetNext:0", shape=(2, 1), dtype=float64)
Tensor("IteratorGetNext:1", shape=(2, 2), dtype=float64)
Tensor("IteratorGetNext:2", shape=(2, 1), dtype=int64)
Tensor("IteratorGetNext:0", shape=(2, 1), dtype=float64)
Tensor("IteratorGetNext:1", shape=(2, 2), dtype=float64)
Tensor("IteratorGetNext:2", shape=(2, 1), dtype=int64)
 9110/34390 [======>.......................] - ETA: 40:42 - loss: 57139.2977 ETA

KeyboardInterrupt: 

# Tests - Works using x_test_df and checkin_data

In [23]:
x_test_df['User'] = x_test_df['User'].astype('int')
x_test_df['User'] = pd.to_numeric(x_test_df['User'])
for index, row in x_test_df.iterrows():
    
    test = checkin_data.loc[
        (checkin_data['user_id'] == x_test_df.loc[index, 'User']) & 
        (checkin_data['latitude'] == x_test_df.loc[index, 'Latitude']) & 
        (checkin_data['longitude'] == x_test_df.loc[index, 'Longitude'])]
    
    if test.empty:
        visited = 0
    else:
        visited = 1
        
    if visited == 0:
        continue
    
    
    long = tf.convert_to_tensor([row['Longitude']],dtype=None, dtype_hint=None, name=None)
    lat = tf.convert_to_tensor([row['Latitude']],dtype=None, dtype_hint=None, name=None)
    user = tf.convert_to_tensor([row['User']],dtype=None, dtype_hint=None, name=None)
    score = model.predict([user, [long, lat]])
    print("Score:", score[0][0])
    if visited == 1:
        print("The user has visited the PoI")
    else:
        print("The user has not visited the PoI \n")
    
    
    


Score: 123.0889
The user has visited the PoI
Score: 25.664396
The user has visited the PoI
Score: 194.60417
The user has visited the PoI
Score: -126.65549
The user has visited the PoI
Score: 67.29813
The user has visited the PoI
Score: -86.97806
The user has visited the PoI
Score: -86.963356
The user has visited the PoI
Score: -242.43178
The user has visited the PoI
Score: -5.513416
The user has visited the PoI
Score: -242.22806
The user has visited the PoI
Score: -143.27936
The user has visited the PoI
Score: 229.01755
The user has visited the PoI
Score: 229.00754
The user has visited the PoI
Score: 107.062004
The user has visited the PoI
Score: -5.677358
The user has visited the PoI
Score: 123.14945
The user has visited the PoI
Score: -171.49234
The user has visited the PoI
Score: 85.168594
The user has visited the PoI
Score: 1.3155111
The user has visited the PoI
Score: -126.538124
The user has visited the PoI
Score: 1.3146948
The user has visited the PoI
Score: 154.58261
The user h